<a href="https://colab.research.google.com/github/BrandonACS/ModeloCovid/blob/main/Modelo_para_predecir_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [3]:
!wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-09 19:00:50--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18045440 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip.1’

datos_abiertos_covi 100%[===================>]  17.21M  7.60MB/s    in 2.3s    

2023-11-09 19:00:53 (7.60 MB/s) - ‘datos_abiertos_covid19.zip.1’ saved [18045440/18045440]



In [4]:
!unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
replace COVID19MEXICO.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
import numpy as np
import pandas as pd


data=pd.read_csv('COVID19MEXICO.csv',low_memory=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   FECHA_ACTUALIZACION    1186694 non-null  object
 1   ID_REGISTRO            1186694 non-null  object
 2   ORIGEN                 1186694 non-null  int64 
 3   SECTOR                 1186694 non-null  int64 
 4   ENTIDAD_UM             1186694 non-null  int64 
 5   SEXO                   1186694 non-null  int64 
 6   ENTIDAD_NAC            1186694 non-null  int64 
 7   ENTIDAD_RES            1186694 non-null  int64 
 8   MUNICIPIO_RES          1186694 non-null  int64 
 9   TIPO_PACIENTE          1186694 non-null  int64 
 10  FECHA_INGRESO          1186694 non-null  object
 11  FECHA_SINTOMAS         1186694 non-null  object
 12  FECHA_DEF              1186694 non-null  object
 13  INTUBADO               1186694 non-null  int64 
 14  NEUMONIA               1186694 non

In [6]:
df=data

df = df.drop(columns=['ID_REGISTRO','ORIGEN','SECTOR','ENTIDAD_UM','SEXO','ENTIDAD_NAC','ENTIDAD_RES','MUNICIPIO_RES','NACIONALIDAD','HABLA_LENGUA_INDIG','INDIGENA','MIGRANTE','PAIS_NACIONALIDAD','PAIS_ORIGEN'])
df.columns

df['FECHA_ACTUALIZACION']=pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO']=pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS']=pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF']=pd.to_datetime(df['FECHA_DEF'],errors='coerce')


for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)


from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()


for col in ['TIPO_PACIENTE']:
  df[col]=le.fit_transform(df[col])
# crear caracteristicas apartir de las fechas

df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']- df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION']- df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']- df['FECHA_SINTOMAS']).dt.days


# Red NUERONAL REVISAR ESTO
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)



df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF','DIAS_DEFUNCION'])

X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   TIPO_PACIENTE          1186694 non-null  int64
 1   INTUBADO               1186694 non-null  int64
 2   NEUMONIA               1186694 non-null  int64
 3   EDAD                   1186694 non-null  int64
 4   EMBARAZO               1186694 non-null  int64
 5   DIABETES               1186694 non-null  int64
 6   EPOC                   1186694 non-null  int64
 7   ASMA                   1186694 non-null  int64
 8   INMUSUPR               1186694 non-null  int64
 9   HIPERTENSION           1186694 non-null  int64
 10  OTRA_COM               1186694 non-null  int64
 11  CARDIOVASCULAR         1186694 non-null  int64
 12  OBESIDAD               1186694 non-null  int64
 13  RENAL_CRONICA          1186694 non-null  int64
 14  TABAQUISMO             1186694 non-null  int64
 15

In [9]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

cuml_RFC=RandomForestClassifier(random_state=42)
cuml_KNC=KNeighborsClassifier(n_neighbors=10)

classifiers=[('Ramdom Forest',cuml_RFC),('KNN',cuml_KNC)]


for name,clf in classifiers:
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(f'Modelo resultados')
  print(classification_report(y_test,y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


Modelo resultados


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.92      0.01      0.03      1673
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     84200
           4       0.00      0.00      0.00        35
           5       0.60      0.25      0.35       396
           6       0.94      0.97      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.64      0.46      0.48    237339
weighted avg       0.99      0.99      0.99    237339

Modelo resultados
              precision    recall  f1-score   support

           1       0.73      0.01      0.03      1673
           2       1.00      0.20      0.33         5
           3       0.95      0.85      0.90     84200
           4       0.00      0.00      0.00        35
           5       0.00      0.00      0.00       396
           6       0.96      0.92      0.94     12816
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [11]:
!pip install gradio

In [21]:
import gradio as gr
import pandas as pd
import numpy as np


def opcions(pregunta):
  if(pregunta == 'SI'):
    pregunta = 0
  elif(pregunta == 'NO'):
    pregunta = 1
  elif(pregunta == 'NO APLICA'):
    pregunta = 97
  elif(pregunta == 'SE IGNORA'):
    pregunta = 98
  elif(pregunta == 'NO ESPECIFICADO'):
    pregunta = 99
  return pregunta

def predict(tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):
    if(tipo_paciente == 'AMBULATORIO'):
      tipo_paciente = 0
    elif(tipo_paciente == 'HOSPITALIZADO'):
      tipo_paciente = 1
    if(resultado_lab == 'POSITIVO A SARS-COV-2'):
      resultado_lab = 1
    elif(resultado_lab == 'NEGATIVO A SARS-COV-2'):
      resultado_lab = 2
    elif(resultado_lab == 'NO RESULTADO PENDIENTE'):
      resultado_lab = 3
    elif(resultado_lab == 'RESULTADO NO ADECUADO'):
      resultado_lab = 4
    elif(resultado_lab ==  'NO APLICA'):
      resultado_lab = 97
    if(resultado_antigeno == 'POSITIVO A SARS-COV-2'):
      resultado_antigeno = 1
    elif(resultado_antigeno == 'NEGATIVO A SARS-COV-2'):
      resultado_antigeno = 2
    elif(resultado_antigeno ==  'NO APLICA'):
      resultado_antigeno = 97
    intubado = opcions(intubado)
    neumonia = opcions(neumonia)
    embarazo = opcions(embarazo)
    epoc = opcions(epoc)
    asma = opcions(asma)
    inmusupr = opcions(inmusupr)
    hipertension = opcions(hipertension)
    otra_com = opcions(otra_com)
    cardiovascular = opcions(cardiovascular)
    obesidad = opcions(obesidad)
    renal_cronica = opcions(renal_cronica)
    tabaquismo = opcions(tabaquismo)
    otro_caso = opcions(otro_caso)
    toma_muestra_lab = opcions(toma_muestra_lab)
    toma_muestra_antigeno = opcions(toma_muestra_antigeno)
    uci = opcions(uci)
    diabetes = opcions(diabetes)
    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[ tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[ 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)

    prediction=prediction.item()

    if(prediction == 1.0):
      prediction= 'CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA'
    elif(prediction == 2.0):
      prediction= 'CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN'
    elif(prediction == 3.0):
      prediction= 'CASO DE SARS-COV-2  CONFIRMADO'
    elif(prediction == 4.0):
      prediction= 'INVÁLIDO POR LABORATORIO'
    elif(prediction == 5.0):
      prediction= 'NO REALIZADO POR LABORATORIO'
    elif(prediction == 6.0):
      prediction= 'CASO SOSPECHOSO'
    elif(prediction == 7.0):
      prediction= 'NEGATIVO A SARS-COV-2'
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=['AMBULATORIO', 'HOSPITALIZADO'], label='Tipo de Paciente'),
    gr.Radio(choices=['SI', 'NO', 'NO APLICA', 'SE IGNORA', 'NO ESPECIFICADO'], label='Intubado'),
    gr.Radio(choices=['SI', 'NO', 'NO APLICA', 'SE IGNORA', 'NO ESPECIFICADO'], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=['SI', 'NO', 'NO APLICA', 'SE IGNORA', 'NO ESPECIFICADO'], label='Embarazo'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Diabetes'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='EPOC'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Asma'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Inmunosupresión'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Hipertensión'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Otra Comorbilidad'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Cardiovascular'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Obesidad'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Tabaquismo'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Contacto con otro caso'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=['POSITIVO A SARS-COV-2','NEGATIVO A SARS-COV-2','NO RESULTADO PENDIENTE','RESULTADO NO ADECUADO', 'NO APLICA'], label='Resultado de laboratorio'),
    gr.Radio(choices=['SI', 'NO','SE IGNORA'], label='Toma de muestra de antígeno'),
    gr.Radio(choices=['POSITIVO A SARS-COV-2','NEGATIVO A SARS-COV-2', 'NO APLICA'], label='Resultado de antígeno'),
    gr.Radio(choices=['SI', 'NO', 'NO APLICA', 'SE IGNORA', 'NO ESPECIFICADO'], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e20567999a81e71f98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e20567999a81e71f98.gradio.live
